In [3]:
import os
import numpy as np
from edpyt.espace import build_espace, screen_espace
from edpyt.gf2_lanczos import build_gf2_lanczos
from scipy.optimize import minimize
new_path = '/capstor/scratch/cscs/ajayaraj/qtpyt-tests/pentadienyl'
os.chdir(new_path)

In [4]:
class Sigma:
    def __init__(self, gf, H_eff, eta=1e-5):
        self.gf = gf
        self.eta = eta
        self.H_eff = H_eff

    def retarded(self, energy):
        energies = np.atleast_1d(energy)
        g = self.gf(energies, self.eta)
        sigma = np.empty((energies.size, self.gf.n, self.gf.n), complex)
        for e, energy in enumerate(energies):
            sigma[e] = energy - self.H_eff - np.linalg.inv(g[..., e])
        return sigma

In [5]:
# === Load inputs ===
input_folder = "output/lowdin"
output_folder = "output/lowdin/beta_1000/ed"
os.makedirs(output_folder, exist_ok=True)

H_eff = np.load(f"{input_folder}/effective_hamiltonian.npy")
occupancy_goal = np.load(f"{input_folder}/beta_1000/occupancies.npy")
V = np.loadtxt(f"{input_folder}/U_matrix.txt")

# === Parameters ===
nimp = H_eff.shape[0]
eta = 1e-3
beta = 1000
energies = np.array([-1000.0, 1000.0])

# === Initial double counting ===
DC0 = np.diag(V.diagonal() * (occupancy_goal - 0.5))
neig = np.ones((nimp + 1) * (nimp + 1), int) * 4


In [6]:
# === Objective function ===
def F(double_counting_diag):
    DC = np.diag(double_counting_diag)
    espace, egs = build_espace(H_eff - DC, V, neig_sector=neig)
    screen_espace(espace, egs, beta)
    gf = build_gf2_lanczos(H_eff - DC, V, espace, beta, egs)
    sigma = Sigma(gf, H_eff, eta=eta)
    sig = sigma.retarded(energies)
    sig_real_diag = sig.real.diagonal(axis1=1, axis2=2)  # shape (2, nimp)
    cost = np.linalg.norm(sig_real_diag) / sig_real_diag.shape[1]
    print(f"[F] Cost: {cost:.6f}, DC_diag: {double_counting_diag}")
    return cost

In [7]:
# === Minimize ===
res = minimize(
    F,
    DC0.diagonal(),
    method="BFGS",
    options={"disp": True, "eps": 5.0},
)


[F] Cost: 2.567404, DC_diag: [5.46515887 1.97886514 2.52035611 2.22202704 2.7152016  2.12250731
 5.90703705]
[F] Cost: 2.625291, DC_diag: [10.46515887  1.97886514  2.52035611  2.22202704  2.7152016   2.12250731
  5.90703705]
[F] Cost: 2.241490, DC_diag: [5.46515887 6.97886514 2.52035611 2.22202704 2.7152016  2.12250731
 5.90703705]
[F] Cost: 3.243522, DC_diag: [5.46515887 1.97886514 7.52035611 2.22202704 2.7152016  2.12250731
 5.90703705]
[F] Cost: 3.331148, DC_diag: [5.46515887 1.97886514 2.52035611 7.22202704 2.7152016  2.12250731
 5.90703705]
[F] Cost: 3.356275, DC_diag: [5.46515887 1.97886514 2.52035611 2.22202704 7.7152016  2.12250731
 5.90703705]
[F] Cost: 2.294710, DC_diag: [5.46515887 1.97886514 2.52035611 2.22202704 2.7152016  7.12250731
 5.90703705]


KeyboardInterrupt: 